In [362]:
import pandas as pd
import numpy as np 
import csv
import re
from difflib import SequenceMatcher

### **Problem 01:**

In [363]:
file_input = '../Data/temp_data.csv'
file_output = '../Data/clean_data.csv'

In [364]:
# Hàm sửa lỗi dư hoặc thiếu dấu ngoặc kép trong file CSV
def fix_csv_quotes(file_input, file_output):
    # Mở file đầu vào để đọc và file đầu ra để ghi
    with open(file_input, 'r', encoding = 'utf-8') as infile, open(file_output, 'w', newline = '', encoding = 'utf-8') as outfile:
        # Đọc tất cả các dòng từ file đầu vào
        reader = infile.readlines()
        # Tạo writer để ghi vào file đầu ra
        writer = csv.writer(outfile)

        # Lặp qua từng dòng trong file đầu vào
        for line in reader:
            # Kiểm tra số lượng dấu ngoặc kép trong dòng
            num_quotes = line.count('"')
            # Nếu số lượng dấu ngoặc kép là số lẻ, thêm một dấu ngoặc kép ở cuối dòng
            if num_quotes % 2 != 0:
                line = line.strip() + '"\n'
            # Ghi lại dòng đã sửa vào file đầu ra
            writer.writerow(next(csv.reader([line], skipinitialspace = True)))

In [365]:
fix_csv_quotes(file_input, file_output)

### **Problem 02:**

In [366]:
column_index = 8  # Chỉ số của cột cần làm sạch (bắt đầu từ 0), cột 8 là cột Description

def clean_string(string):

    """
    Hàm này sẽ làm sạch chuỗi bằng cách loại bỏ các biểu tượng và ký tự không cần thiết.
    (nhưng giữ lại các kí tự chữ (hoa và thường), số, dấu chấm, dấu phẩy, dấu | )
    
    string: Chuỗi cần làm sạch.
    cleanString: Chuỗi đã được làm sạch.

    """

    # Hàm isinstance check xem data input của mình có phải là string không 
    if isinstance(string, str):
        cleanString = re.sub(r'[^\w\s.,|]', '', string) # Xoá tất cả các kí tự ngoại trừ chữ (thường, hoa), số, dấu chấm, dấu phẩy, dấu |
        return cleanString
    else:
        return string

In [367]:
def clean_file_csv(file_input, file_output, column_index):

    """
    Hàm này sẽ làm sạch một cột cụ thể trong file CSV, do có những cột khác (VD cột Postdate có dấu : ), 
    Vì vậy nếu apply full file csv nó sẽ fix hết cả những cột không cần thiết -> chỉ áp dụng chính xác cột cần xử lí thôi.

    input_csv (str): Đường dẫn tới file CSV đầu vào (temp_data).
    output_csv (str): Đường dẫn tới file CSV đầu ra (clean_data).
    column_index (int): Chỉ số của cột cần làm sạch.
    """

    # Đọc file CSV vào DataFrame
    df = pd.read_csv(file_input, on_bad_lines = 'skip')

    # Kiểm tra xem chỉ số cột có hợp lệ không
    if column_index < len(df.columns):
        # Áp dụng hàm làm sạch cho cột cụ thể
        col = df.columns[column_index]
        df[col] = df[col].apply(clean_string)
    else:
        print(f"Invalid column index !")

    # Lưu lại DataFrame đã được làm sạch thành file CSV mới
    df.to_csv(file_output, index = False)



In [368]:
clean_file_csv(file_input, file_output, column_index)

### **Problem 03:**

In [369]:
# Đọc lại file csv, xoá các dòng bị lỗi 
# (mình nhận thấy số lượng dòng bị lỗi không nhiều ~ 10 dòng nên mình sẽ skip luôn vì tụi mình có nhiều data mà =))))
df = pd.read_csv('../Data/clean_data.csv', on_bad_lines = 'skip')

# Loại bỏ cột No và cột Title không cần thiết 
df = df.drop(columns = df.columns[[0, 2]])

# Thay thế "Thoả thuận" bằng NaN cho dễ xử lí về sau (tạm thời)
df['Price'] = df['Price'].replace('TT', np.nan)

# Thiết lập ngưỡng cho số lượng giá trị NaN tối đa cho phép trong một hàng
max_nan = 3  

# Xóa các hàng có số lượng giá trị NaN lớn hơn ngưỡng
df = df.dropna(thresh = len(df.columns) - max_nan)  # thresh: chỉ định số lượng giá trị NOT NULL cần thiết để giữ lại hàng

# Xử lí cột Price 
# df = df[df['Price'] <= 1000]
df['Price'] = df['Price'].astype(float)
df['Price'] = df['Price'].apply(lambda x: x / 1e9 if x > 1e9 else x / 1000 if x > 1000 else x)
df['Price'] = df['Price'].round(2)


# Xử lí cột Postdate (chỉ giữ lại Ngày xoá Giờ)
# Chuyển đổi cột 'Postdate' sang kiểu datetime
df['Postdate'] = pd.to_datetime(df['Postdate'])

# Tạo một cột mới chỉ chứa ngày
df['Date'] = df['Postdate'].dt.date

# Tìm vị trí (index) của cột 'Postdate'
col_index = df.columns.get_loc('Postdate')

# Xóa cột 'Postdate' cũ 
df = df.drop(columns = ['Postdate'])

# Chèn cột 'Date' mới vào đúng vị trí cũ của cột 'Postdate'
df.insert(col_index, 'Postdate', df.pop('Date'))

# Load lại những thay đổi vào file clean_data.csv
df.to_csv(file_output, index = False)


In [370]:
def remove_unusual_line_terminators(file_input, file_output):
    with open(file_input, 'r', encoding='utf-8') as infile:
        content = infile.read()

    # Loại bỏ các ký tự kết thúc dòng không bình thường (LS và PS)
    cleaned_content = content.replace('\u2028', '\n').replace('\u2029', '\n')

    with open(file_output, 'w', encoding='utf-8', newline='') as outfile:
        outfile.write(cleaned_content)

In [371]:
# Hàm loại bỏ các dòng trùng lặp trong file CSV
def remove_duplicates(df):  
    # Đảm bảo rằng các giá trị trong 'Postdate' và 'Description' là chuỗi
    df['Description'] = df['Description'].astype(str)
    
    # Bước 1: Loại bỏ các dòng trùng lặp mà tất cả các cột đều giống nhau
    df = df.drop_duplicates()
    
    # Bước 2: Nhóm các dòng theo các cột 'Price', 'Area', 'Bedrooms', 'WCs', 'Address'
    grouped = df.groupby(['Price', 'Area', 'Bedrooms', 'WCs', 'Address'])
    
    # Danh sách các chỉ số của các dòng cần giữ lại
    indices_to_keep = set(df.index)

    def similar(a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    # Lặp qua từng nhóm để kiểm tra sự tương đồng
    for _, group in grouped:
        group_indices = list(group.index)
        for i in range(len(group_indices)):
            for j in range(i + 1, len(group_indices)):
                idx1, idx2 = group_indices[i], group_indices[j]
                row1, row2 = df.loc[idx1], df.loc[idx2]
                if row1['Postdate'] == row2['Postdate'] and similar(row1['Description'], row2['Description']) >= 0.8:
                    indices_to_keep.discard(idx2)
    
    # Chuyển đổi tập hợp các chỉ số cần giữ lại thành danh sách
    indices_to_keep = list(indices_to_keep)
    
    # Lọc lại DataFrame với các chỉ số cần giữ lại
    df = df.loc[indices_to_keep].reset_index(drop=True)
    
    # Trả về DataFrame đã loại bỏ các dòng trùng lặp
    return df

In [372]:
# Áp dụng hàm remove_duplicates lên DataFrame đã được làm sạch
df_cleaned = remove_duplicates(df)
# Tạo lại cột No
df_cleaned.insert(0, 'No', range(1, len(df_cleaned) + 1))

# Lưu lại DataFrame đã làm sạch và loại bỏ trùng lặp vào file CSV
df_cleaned.to_csv(file_output, index=False)

In [373]:
# # Hàm tính tỷ lệ tương đồng giữa hai chuỗi
# def similar(a, b):
#     return SequenceMatcher(None, a, b).ratio()

# # Hàm loại bỏ các dòng trùng lặp trong file CSV
# def remove_duplicates(file_input):
#     # Đọc dữ liệu từ file CSV
#     df = pd.read_csv(file_input, on_bad_lines='skip', delimiter=',')
    
#     # Đảm bảo rằng các giá trị trong 'Postdate' và 'Description' là chuỗi
#     #df['Postdate'] = df['Postdate'].astype(str)
#     df['Description'] = df['Description'].astype(str)
    
#     # Bước 1: Loại bỏ các dòng trùng lặp mà tất cả các cột đều giống nhau
#     df = df.drop_duplicates()
    
#     # Bước 2: Loại bỏ các dòng trùng lặp mà các cột cụ thể giống nhau và 'Postdate' và 'Description' giống nhau từ 80% trở lên
#     def is_similar(row1, row2):
#         return (
#                 row1['Price'] == row2['Price'] 
#                 and row1['Area'] == row2['Area'] 
#                 and row1['Bedrooms'] == row2['Bedrooms'] 
#                 and row1['WCs'] == row2['WCs'] 
#                 and row1['Address'] == row2['Address'] 
#                 #and similar(row1['Postdate'], row2['Postdate']) >= 0.8 
#                 and row1['Postdate'] == row2['Postdate']
#                 and similar(row1['Description'], row2['Description']) >= 0.8
#                 )
    
#     # Danh sách các chỉ số của các dòng cần loại bỏ
#     indices_to_remove = []
#     # Lặp qua từng cặp dòng trong DataFrame để kiểm tra sự tương đồng
#     for i in range(len(df)):
#         for j in range(i + 1, len(df)):
#             if is_similar(df.iloc[i], df.iloc[j]):
#                 indices_to_remove.append(j)
    
#     # Loại bỏ các dòng duplicate theo chỉ số đã xác định
#     df = df.drop(indices_to_remove).reset_index(drop=True)
    
#     # Trả về DataFrame đã loại bỏ các dòng trùng lặp
#     return df
